In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Tables

### Links: 
[[rep_mobile_citizens_id_city_partner]]

[[subscriptions_st_mobile_ch]]

In [ ]:
query_text = """--sql
    CREATE TABLE db1.t_full_citizen_id_in_flat_with_subscriptions
    (
        `report_date` Date,
        `address_uuid` String,
        `citizen_id` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)

In [ ]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_full_citizen_id_in_flat_with_subscriptions_mv
REFRESH EVERY 1 DAY OFFSET 4 HOUR 25 MINUTE TO db1.t_full_citizen_id_in_flat_with_subscriptions AS 
WITH t1 AS (SELECT 
	rmcicp.report_date AS report_date,
	flat_uuid
FROM db1.rep_mobile_citizens_id_city_partner AS rmcicp
JOIN db1.subscriptions_st_mobile_ch AS ssm 
	ON rmcicp.report_date = ssm.report_date 
	AND rmcicp.citizen_id = ssm.citizen_id
WHERE  ssm.state = 'activated'
)
SELECT
	t1.report_date AS report_date,
	rmcicp.address_uuid AS address_uuid,
	rmcicp.citizen_id AS citizen_id
FROM db1.rep_mobile_citizens_id_city_partner AS rmcicp
JOIN t1 AS ssm 
	ON rmcicp.report_date = ssm.report_date 
	AND rmcicp.flat_uuid = ssm.flat_uuid
WHERE t1.flat_uuid !=''
    """
ch.query_run(query_text)

___
## Tools
___
### query

In [2]:
query_text = """
    SELECT
        *
    FROM db1.t_full_citizen_id_in_flat_with_subscriptions
    order by report_date desc
    limit 10
    """
ch.query_run(query_text)

report_date,address_uuid,citizen_id
date,str,i64
2025-09-06,"""001cf013-43a8-4cc7-906a-4e1b7a…",427330
2025-09-06,"""001cf013-43a8-4cc7-906a-4e1b7a…",427261
2025-09-06,"""004601ff-2956-475e-88fa-227296…",211520
2025-09-06,"""000c4f15-2478-4a10-a87e-b962c9…",109198
2025-09-06,"""001cf013-43a8-4cc7-906a-4e1b7a…",427165
2025-09-06,"""000c4f15-2478-4a10-a87e-b962c9…",102325
2025-09-06,"""001cf013-43a8-4cc7-906a-4e1b7a…",427246
2025-09-06,"""001cf013-43a8-4cc7-906a-4e1b7a…",427158
2025-09-06,"""001cf013-43a8-4cc7-906a-4e1b7a…",425922


### drop table

In [ ]:
query_text = """
    DROP TABLE db1.t_full_citizen_id_in_flat_with_subscriptions
    """
ch.query_run(query_text)

### drop mv

In [ ]:
query_text = """
    DROP TABLE db1.t_full_citizen_id_in_flat_with_subscriptions_mv
    """
ch.query_run(query_text)


### SYSTEM REFRESH

In [2]:
query_text = """
SYSTEM REFRESH VIEW db1.t_full_citizen_id_in_flat_with_subscriptions_mv
"""

ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,649053,"""657f9526-f67b-4623-90ca-a29a60…"
